- [gevent For the Working Python Developer](http://sdiehl.github.io/gevent-tutorial/) (En)
- [Python 개발자를 위한 gevent](http://leekchan.com/gevent-tutorial-ko/) (Ko)

In [1]:
# yiedling for context switch with gevent.sleep(0)
import gevent

def foo():
    print('Running in foo')
    gevent.sleep(0)
    print('Explicit context switch to foo again')
    
def bar():
    print('Explicit context to bar')
    gevent.sleep(0)
    print('Implicit context switch back to bar')
    
gevent.joinall([
    gevent.spawn(foo),
    gevent.spawn(bar),
])

Running in foo
Explicit context to bar
Explicit context switch to foo again
Implicit context switch back to bar


[<Greenlet at 0x7fdc202983d8>, <Greenlet at 0x7fdc20298508>]

In [2]:
# blocking call using select()

import time
import gevent
from gevent import select

start = time.time()
tic = lambda: 'at %1.1f seconds' % (time.time() - start)

def gr1():
    # Busy waits for a second, but we don't want to stick around...
    print('Started Polling: %s' % tic())
    select.select([], [], [], 2)
    print('Ended Polling: %s' % tic())
    
def gr2():
    # Busy waits for a second, but we don't want to stick around...
    print('Started Polling: %s' % tic())
    select.select([], [], [], 2)
    print('Ended Polling: %s' % tic())
    
def gr3():
    print('Hey lets do some stuff while the greenlets poll, %s' % tic())
    gevent.sleep(1)
    
gevent.joinall([
    gevent.spawn(gr1),
    gevent.spawn(gr2),
    gevent.spawn(gr3),
])

Started Polling: at 0.0 seconds
Started Polling: at 0.0 seconds
Hey lets do some stuff while the greenlets poll, at 0.0 seconds
Ended Polling: at 2.0 seconds
Ended Polling: at 2.0 seconds


In [3]:
# non-deterministic task
import gevent
import random

def task(pid):
    """
    some non-deterministic task
    """
    gevent.sleep(random.randint(0,2) * 0.001)
    print('Task %s done' % pid)
    
def synchronous():
    for i in range(1, 10):
        task(i)
        
def asynchronous():
    threads = [gevent.spawn(task, i) for i in range(10)]
    gevent.joinall(threads)
    
print('Synchronous:')
synchronous()

print('Asynchronous:')
asynchronous()

Synchronous:
Task 1 done
Task 2 done
Task 3 done
Task 4 done
Task 5 done
Task 6 done
Task 7 done
Task 8 done
Task 9 done
Asynchronous:
Task 3 done
Task 5 done
Task 0 done
Task 1 done
Task 2 done
Task 8 done
Task 9 done
Task 4 done
Task 6 done
Task 7 done


In [4]:
# synchronous vs asynchronous
import gevent.monkey
gevent.monkey.patch_socket()

import gevent
from urllib.request import urlopen
import simplejson as json


def fetch(pid):
    response = urlopen('http://date.jsontest.com/')
    result = response.read()
    json_result = json.loads(result)
    datetime = json_result['time']
    
    print('Process %s: %s' % (pid, datetime))
    return json_result['time']

def synchronous():
    for i in range(1,10):
        fetch(i)

def asynchronous():
    threads = []
    for i in range(1,10):
        threads.append(gevent.spawn(fetch, i))
    gevent.joinall(threads)

print('Synchronous:')
synchronous()

print('Asynchronous:')
asynchronous()

Synchronous:
Process 1: 11:35:14 AM
Process 2: 11:35:14 AM
Process 3: 11:35:16 AM
Process 4: 11:35:16 AM
Process 5: 11:35:17 AM
Process 6: 11:35:18 AM
Process 7: 11:35:18 AM
Process 8: 11:35:19 AM
Process 9: 11:35:20 AM
Asynchronous:
Process 6: 11:35:21 AM
Process 5: 11:35:21 AM
Process 2: 11:35:21 AM
Process 4: 11:35:21 AM
Process 9: 11:35:21 AM
Process 1: 11:35:21 AM
Process 8: 11:35:21 AM
Process 7: 11:35:21 AM
Process 3: 11:35:21 AM


In [5]:
# deterministic greenlet
import time

def echo(i):
    time.sleep(0.001)
    return i

# Non Deterministic Process Pool

from multiprocessing.pool import Pool

p = Pool(10)
run1 = [a for a in p.imap_unordered(echo, range(10))]
run2 = [a for a in p.imap_unordered(echo, range(10))]
run3 = [a for a in p.imap_unordered(echo, range(10))]
run4 = [a for a in p.imap_unordered(echo, range(10))]

print(run1 == run2 == run3 == run4)

# Deterministic Gevent Pool

from gevent.pool import Pool

p = Pool(10)
run1 = [a for a in p.imap_unordered(echo, range(10))]
run2 = [a for a in p.imap_unordered(echo, range(10))]
run3 = [a for a in p.imap_unordered(echo, range(10))]
run4 = [a for a in p.imap_unordered(echo, range(10))]

print(run1 == run2 == run3 == run4)

False
True


In [6]:
# initializing Greenlet 1

import gevent
from gevent import Greenlet

def foo(message, n):
    """
    Each thread will be passed the message, and n arguments
    in its initialization.
    """
    gevent.sleep(n)
    print(message)
    
# Initialize a new Greenlet instance running the named function
# foo
thread1 = Greenlet.spawn(foo, 'Hello', 2)

# Wrapper for creating and running a new Greenlet from the named
# function foo, with the passed arguments
thread2 = gevent.spawn(foo, 'I live!', 1)

# Lambda expressions
thread3 = gevent.spawn(lambda x: (x+1), 2)

threads = [thread1, thread2, thread3]

# Block until all threads complete.
gevent.joinall(threads)

I live!
Hello


In [7]:
# initializing Greenlet 2: extending Greenlet

import gevent
from gevent import Greenlet

class MyGreenlet(Greenlet):
    
    def __init__(self, message, n):
        Greenlet.__init__(self)
        self.message = message
        self.n = n
        
    def _run(self):
        print(self.message)
        gevent.sleep(self.n)
        
g = MyGreenlet('Hi There!', 3)
g.start()
g.join()

Hi There!


In [8]:
# Greenlet State
# - started
# - ready
# - successful
# - value
# - exception
import gevent

def win():
    return 'You win!'

def fail():
    raise Exception('You fail at failing')
    
winner = gevent.spawn(win)
loser = gevent.spawn(fail)

print(winner.started)
print(loser.started)

try:
    gevent.joinall([winner, loser])
except Exception as e:
    print('This will never be reached')
    
print(winner.value)
print(loser.value)

print(winner.ready())
print(loser.ready())

print(winner.successful())
print(loser.successful())

print(loser.exception)

True
True
You win!
None
True
True
True
False
You fail at failing


Traceback (most recent call last):
  File "/home/luno/.pyenv/versions/3.6.0/envs/jupyter-env/lib/python3.6/site-packages/gevent/greenlet.py", line 536, in run
    result = self._run(*self.args, **self.kwargs)
  File "<ipython-input-8-3384d5694d5e>", line 13, in fail
    raise Exception('You fail at failing')
Exception: You fail at failing
Thu Feb 16 20:35:39 2017 <Greenlet at 0x7fdc18adbf20: fail> failed with Exception



In [9]:
import gevent
import signal

def run_forever():
    gevent.sleep(1000)
    
gevent.signal(signal.SIGINT, gevent.kill)
thread = gevent.spawn(run_forever)
thread.join()

Traceback (most recent call last):
  File "/home/luno/.pyenv/versions/3.6.0/envs/jupyter-env/lib/python3.6/site-packages/gevent/hub.py", line 279, in handle
    self.handler(*self.args, **self.kwargs)
TypeError: kill() missing 1 required positional argument: 'greenlet'
Thu Feb 16 20:35:57 2017


TypeError: kill() missing 1 required positional argument: 'greenlet'

In [10]:
import gevent
from gevent import Timeout

seconds = 5

timeout = Timeout(seconds)
timeout.start()

def wait():
    gevent.sleep(10)

try:
    gevent.spawn(wait).join()
except Timeout:
    print('Could not complete')

Could not complete


In [11]:
import gevent
from gevent import Timeout

time_to_wait = 5

class TooLong(Exception):
    pass

with Timeout(time_to_wait, TooLong):
    gevent.sleep(10)

TooLong: 

In [12]:
import gevent
from gevent import Timeout

def wait():
    gevent.sleep(2)
    
timer = Timeout(1).start()
thread1 = gevent.spawn(wait)

try:
    thread1.join(timeout=timer)
except Timeout:
    print('Thread 1 timed out')
    
timer = Timeout.start_new(1)
thread2 = gevent.spawn(wait)

try:
    thread2.get(timeout=timer)
except Timeout:
    print('Thread 2 timed out')
    
try:
    gevent.with_timeout(1, wait)
except Timeout:
    print('Thread 3 timed out')

Thread 1 timed out
Thread 2 timed out
Thread 3 timed out


In [14]:
#import socket
socket.socket = __import__('socket')
print(socket.socket)

print('After monkey patch')
from gevent import monkey
monkey.patch_socket()
print(socket.socket)

import select
print(select.select)
monkey.patch_select()
print("After monkey patch")
print(select.select)

"""
<class 'socket.socket'>
After monkey patch
<class 'gevent._socket3.socket'>
<built-in function select>
After monkey patch
<function select at 0x7fa8c020e9d8>
"""

<module 'socket' from '/home/luno/.pyenv/versions/3.6.0/lib/python3.6/socket.py'>
After monkey patch
<class 'gevent._socket3.socket'>
<function select at 0x7fdc20036d90>
After monkey patch
<function select at 0x7fdc20036d90>


"\n<class 'socket.socket'>\nAfter monkey patch\n<class 'gevent._socket3.socket'>\n<built-in function select>\nAfter monkey patch\n<function select at 0x7fa8c020e9d8>\n"

In [15]:
import gevent
from gevent.event import Event

evt = Event()

def setter():
    '''After 3 seconds, wake all threads waiting on the value of evt'''
    print('A: Hey wait for me, I have to do something')
    gevent.sleep(3)
    print("Ok, I'm done")
    evt.set()
    
def waiter():
    '''After 3 seconds the get call will unblock'''
    print("I'll wait for you")
    evt.wait()  # blocking
    print("It's about time")
    
gevent.joinall([
        gevent.spawn(setter),
        gevent.spawn(waiter),
        gevent.spawn(waiter),
        gevent.spawn(waiter),
        gevent.spawn(waiter),
        gevent.spawn(waiter)
    ])

A: Hey wait for me, I have to do something
I'll wait for you
I'll wait for you
I'll wait for you
I'll wait for you
I'll wait for you
Ok, I'm done
It's about time
It's about time
It's about time
It's about time
It's about time


In [16]:
import gevent
from gevent.event import AsyncResult
a = AsyncResult()

def setter():
    """
    After 3 seconds set the result of a.
    """
    print('setter')
    gevent.sleep(3)
    a.set('Hello!')

def waiter():
    """
    After 3 seconds the get call will unblock after the setter
    puts a value into the AsyncResult.
    """
    print(a.get())

gevent.joinall([
    gevent.spawn(setter),
    gevent.spawn(waiter),
])

setter
Hello!


[<Greenlet at 0x7fdc180692a8>, <Greenlet at 0x7fdc18069210>]

In [17]:
import gevent
from gevent.queue import Queue

tasks = Queue()

def worker(n):
    while not tasks.empty():
        task = tasks.get()
        print('Worker %s got task %s' % (n, task))
        gevent.sleep(0)
        
    print('Quitting time!')
    
def boss():
    for i in range(1, 25):
        tasks.put_nowait(i)
        
gevent.spawn(boss).join()

gevent.joinall([
    gevent.spawn(worker, 'steve'),
    gevent.spawn(worker, 'john'),
    gevent.spawn(worker, 'nancy'),
])

Worker steve got task 1
Worker john got task 2
Worker nancy got task 3
Worker steve got task 4
Worker john got task 5
Worker nancy got task 6
Worker steve got task 7
Worker john got task 8
Worker nancy got task 9
Worker steve got task 10
Worker john got task 11
Worker nancy got task 12
Worker steve got task 13
Worker john got task 14
Worker nancy got task 15
Worker steve got task 16
Worker john got task 17
Worker nancy got task 18
Worker steve got task 19
Worker john got task 20
Worker nancy got task 21
Worker steve got task 22
Worker john got task 23
Worker nancy got task 24
Quitting time!
Quitting time!
Quitting time!


In [18]:
import gevent
from gevent.queue import Queue, Empty

tasks = Queue(maxsize=3)

def worker(name):
    try:
        while True:
            task = tasks.get(timeout=1)
            print('Worker %s got task %s' % (name, task))
            gevent.sleep(0)
    except Empty:
        print('Quitting time!')
        

def boss():
    """
    Boss will wait to hand out work until a individual worker is
    free since the maxsize of the task queue is 3.
    """
    for i in range(1,10):
        tasks.put(i)
    print('Assigned all work in iteration 1')

    for i in range(10,20):
        tasks.put(i)
    print('Assigned all work in iteration 2')

gevent.joinall([
    gevent.spawn(boss),
    gevent.spawn(worker, 'steve'),
    gevent.spawn(worker, 'john'),
    gevent.spawn(worker, 'bob'),
])

Worker steve got task 1
Worker john got task 2
Worker bob got task 3
Worker steve got task 4
Worker john got task 5
Worker bob got task 6
Assigned all work in iteration 1
Worker steve got task 7
Worker john got task 8
Worker bob got task 9
Worker steve got task 10
Worker john got task 11
Worker bob got task 12
Worker steve got task 13
Worker john got task 14
Worker bob got task 15
Worker steve got task 16
Worker john got task 17
Worker bob got task 18
Assigned all work in iteration 2
Worker steve got task 19
Quitting time!
Quitting time!
Quitting time!


In [19]:
import gevent
from gevent.pool import Group

def talk(msg):
    for i in range(3):
        print(msg)
        
g1 = gevent.spawn(talk, 'bar')
g2 = gevent.spawn(talk, 'foo')
g3 = gevent.spawn(talk, 'fizz')

group = Group()
group.add(g1)
group.add(g2)
group.join()

group.add(g3)
group.join()

bar
bar
bar
foo
foo
foo
fizz
fizz
fizz


True

In [20]:
import gevent
from gevent import getcurrent
from gevent.pool import Group

group = Group()

def hello_from(n):
    print('Size of group %s' % len(group))
    print('Hello from Greenlet %s' % id(getcurrent()))

group.map(hello_from, range(3))

def intensive(n):
    gevent.sleep(3 - n)
    return 'task', n

print('Ordered')

ogroup = Group()
for i in ogroup.imap(intensive, range(3)):
    print(i)

print('Unordered')

igroup = Group()
for i in igroup.imap_unordered(intensive, range(3)):
    print(i)

Size of group 3
Hello from Greenlet 140583272618440
Size of group 3
Hello from Greenlet 140583272618592
Size of group 3
Hello from Greenlet 140583272618744
Ordered
('task', 0)
('task', 1)
('task', 2)
Unordered
('task', 2)
('task', 1)
('task', 0)


In [21]:
# Pool

import gevent
from gevent.pool import Pool

pool = Pool(2)

def hello_from(n):
    print('Size of pool %s' % len(pool))

pool.map(hello_from, range(3))

Size of pool 2
Size of pool 2
Size of pool 1


[None, None, None]

In [22]:
from gevent.pool import Pool

class SocketPool(object):
    
    def __init__(self):
        self.pool = Pool(1000)
        self.pool.start()
        
    def listen(self, socket):
        while True:
            socket.recv()
            
    def add_handler(self, socket):
        if self.pool.full():
            raise Exception("At maximum pool size")
        else:
            self.pool.spawn(self.listen, socket)
            
    def shutdown(self):
        self.pool.kill()

In [23]:
from gevent import sleep
from gevent.pool import Pool
from gevent.lock import BoundedSemaphore

sem = BoundedSemaphore(2)

def worker1(n):
    sem.acquire()
    print('Worker %i acquired semaphore' % n)
    sleep(0)
    sem.release()
    print('Worker %i released semaphore' % n)

def worker2(n):
    with sem:
        print('Worker %i acquired semaphore' % n)
        sleep(0)
    print('Worker %i released semaphore' % n)

pool = Pool()
pool.map(worker1, range(0,2))
pool.map(worker2, range(3,6))

Worker 0 acquired semaphore
Worker 1 acquired semaphore
Worker 0 released semaphore
Worker 1 released semaphore
Worker 3 acquired semaphore
Worker 4 acquired semaphore
Worker 3 released semaphore
Worker 4 released semaphore
Worker 5 acquired semaphore
Worker 5 released semaphore


[None, None, None]

In [24]:
import gevent
from gevent.local import local

stash = local()

def f1():
    stash.x = 1
    print(stash.x)
    
def f2():
    stash.y = 2
    print(stash.y)
    
    try:
        stash.x
    except AttributeError:
        print("x is not local to f2")

g1 = gevent.spawn(f1)
g2 = gevent.spawn(f2)

gevent.joinall([g1, g2])

1
2
x is not local to f2


[<Greenlet at 0x7fdc18069930>, <Greenlet at 0x7fdc18069638>]

In [ ]:
from gevent.local import local
from werkzeug.local import LocalProxy
from werkzeug.wrappers import Request
from contextlib import contextmanager

from gevent.wsgi import WSGIServer

_requests = local()
request = LocalProxy(lambda: _requests.request)

@contextmanager
def sessionmanager(environ):
    _requests.request = Request(environ)
    yield
    _requests.request = None
    
def logic():
    return "Hello " + request.remote_addr

def application(environ, start_response):
    status = '200 OK'

    with sessionmanager(environ):
        body = bytes(logic(), 'utf-8')

    headers = [
        ('Content-Type', 'text/html')
    ]

    start_response(status, headers)
    return [body]

WSGIServer(('', 8000), application).serve_forever()

In [25]:
import gevent
from gevent.subprocess import Popen, PIPE

def cron():
    while True:
        print("cron")
        gevent.sleep(0.2)

g = gevent.spawn(cron)
sub = Popen(['sleep 1; uname'], stdout=PIPE, shell=True)
out, err = sub.communicate()
g.kill()
print(out.rstrip())

cron
cron
cron
cron
cron
cron
b'Linux'


In [27]:
import gevent
from multiprocessing import Process, Pipe
from gevent.socket import wait_read, wait_write

# To Process
a, b = Pipe()

# From Process
c, d = Pipe()

def relay():
    for i in range(10):
        msg = b.recv()
        c.send(msg + " in " + str(i))

def put_msg():
    for i in range(10):
        wait_write(a.fileno())
        a.send('hi')

def get_msg():
    for i in range(10):
        wait_read(d.fileno())
        print(d.recv())

proc = Process(target=relay)
proc.start()

g1 = gevent.spawn(get_msg)
g2 = gevent.spawn(put_msg)
gevent.joinall([g1, g2], timeout=1)

Process Process-22:
Traceback (most recent call last):
  File "/home/luno/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/luno/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-27-0e705d30801d>", line 13, in relay
    msg = b.recv()
  File "/home/luno/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/home/luno/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/luno/.pyenv/versions/3.6.0/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
BlockingIOError: [Errno 11] Resource temporarily unavailable


hi in 0
hi in 1
hi in 2
hi in 3


In [ ]:
import gevent
from gevent import Greenlet
from gevent.queue import Queue

class Actor(Greenlet):
    
    def __init__(self):
        self.inbox = Queue()
        Greenlet.__init__(self)
        
    def receive(self, message):
        raise NotImplemented
        
    def _run(self):
        self.running = True

        while self.running:
            message = self.inbox.get()
            self.receive(message)
            

class Pinger(Actor):
    def receive(self, message):
        print(message)
        pong.inbox.put('ping')
        gevent.sleep(0)


class Ponger(Actor):
    def receive(self, message):
        print(message)
        ping.inbox.put('pong')
        gevent.sleep(0)

ping = Pinger()
pong = Ponger()

ping.start()
pong.start()

ping.inbox.put('start')
gevent.joinall([ping, pong])

In [ ]:
import gevent
import zmq

# Global Context
context = zmq.Context()

def server():
    server_socket = context.socket(zmq.REQ)
    server_socket.bind("tcp://127.0.0.1:5000")

    for request in range(1, 10):
        server_socket.send_string("Hello")
        print('Switched to Server for %s' % str(request))
        # Implicit context switch occurs here
        server_socket.recv()

def client():
    client_socket = context.socket(zmq.REP)
    client_socket.connect("tcp://127.0.0.1:5000")

    for request in range(1, 10):
        client_socket.recv()
        print('Switched to Client for %s' % str(request))
        # Implicit context switch occurs here
        client_socket.send_string("World")

publisher = gevent.spawn(server)
client    = gevent.spawn(client)

gevent.joinall([publisher, client])

In [ ]:
from gevent.server import StreamServer

def handle(socket, address):
    socket.send('Hello from a telnet!\n')
    for i in range(5):
        socket.send(str(i) + '\n')
    socket.close()
    
server = StreamSerer(('127.0.0.1', 5000), handle)
server.server_forever()

In [ ]:
from gevent.wsgi import WSGIServer

def application(environ, start_response):
    status = '200 OK'
    body = '<p>Hello World</p>'

    headers = [
        ('Content-Type', 'text/html')
    ]

    start_response(status, headers)
    return [body]

WSGIServer(('', 8000), application).serve_forever()

In [ ]:
from gevent.pywsgi import WSGIServer

def application(environ, start_response):
    status = '200 OK'

    headers = [
        ('Content-Type', 'text/html')
    ]

    start_response(status, headers)
    yield "<p>Hello"
    yield "World</p>"

WSGIServer(('', 8000), application).serve_forever()

In [ ]:
import gevent
from gevent.queue import Queue, Empty
from gevent.pywsgi import WSGIServer
import simplejson as json

data_source = Queue()

def producer():
    while True:
        data_source.put_nowait('Hello World')
        gevent.sleep(1)
        
def ajax_endpoint(environ, start_response):
    status = '200 OK'
    headers = [
        ('Content-Type', 'application/json')
    ]

    start_response(status, headers)
    
    while True:
        try:
            datum = data_source.get(timeout=5)
            yield json.dumps(datum) + '\n'
        except Empty:
            pass
        
gevent.spawn(producer)

WSGIServer(('', 8000), ajax_endpoint).serve_forever()

In [ ]:
# Websockets
import json
import random

from gevent import pywsgi, sleep
from geventwebsocket.handler import WebSocketHandler

class WebSocketApp(object):
    '''Send random data to the websocket'''
    
    def __call__(self, environ, start_response):
        ws = environ['wsgi.websocket']
        x = 0
        while True:
            data = json.dumps({'x': x, 'y': random.randint(1, 5)})
            ws.send(data)
            x += 1
            sleep(0.5)

server = pywsgi.WSGIServer(("", 10000), WebSocketApp(),
    handler_class=WebSocketHandler)
server.serve_forever()

HTML Page: 
```
<html>
    <head>
        <title>Minimal websocket application</title>
        <script type="text/javascript" src="jquery.min.js"></script>
        <script type="text/javascript">
        $(function() {
            // Open up a connection to our server
            var ws = new WebSocket("ws://localhost:10000/");

            // What do we do when we get a message?
            ws.onmessage = function(evt) {
                $("#placeholder").append('<p>' + evt.data + '</p>')
            }
            // Just update our conn_status field with the connection status
            ws.onopen = function(evt) {
                $('#conn_status').html('<b>Connected</b>');
            }
            ws.onerror = function(evt) {
                $('#conn_status').html('<b>Error</b>');
            }
            ws.onclose = function(evt) {
                $('#conn_status').html('<b>Closed</b>');
            }
        });
    </script>
    </head>
    <body>
        <h1>WebSocket Example</h1>
        <div id="conn_status">Not Connected</div>
        <div id="placeholder" style="width:600px;height:300px;"></div>
    </body>
</html>
```

In [ ]:
# Micro gevent chatroom.
# ----------------------

from flask import Flask, render_template, request

from gevent import queue
from gevent.pywsgi import WSGIServer

import simplejson as json

app = Flask(__name__)
app.debug = True

rooms = {
    'topic1': Room(),
    'topic2': Room(),
}

users = {}

class Room(object):

    def __init__(self):
        self.users = set()
        self.messages = []

    def backlog(self, size=25):
        return self.messages[-size:]

    def subscribe(self, user):
        self.users.add(user)

    def add(self, message):
        for user in self.users:
            print(user)
            user.queue.put_nowait(message)
        self.messages.append(message)

class User(object):

    def __init__(self):
        self.queue = queue.Queue()

@app.route('/')
def choose_name():
    return render_template('choose.html')

@app.route('/<uid>')
def main(uid):
    return render_template('main.html',
        uid=uid,
        rooms=rooms.keys()
    )

@app.route('/<room>/<uid>')
def join(room, uid):
    user = users.get(uid, None)

    if not user:
        users[uid] = user = User()

    active_room = rooms[room]
    active_room.subscribe(user)
    print('subscribe %s %s' % (active_room, user))

    messages = active_room.backlog()

    return render_template('room.html',
        room=room, uid=uid, messages=messages)

@app.route("/put/<room>/<uid>", methods=["POST"])
def put(room, uid):
    user = users[uid]
    room = rooms[room]

    message = request.form['message']
    room.add(':'.join([uid, message]))

    return ''

@app.route("/poll/<uid>", methods=["POST"])
def poll(uid):
    try:
        msg = users[uid].queue.get(timeout=10)
    except queue.Empty:
        msg = []
    return json.dumps(msg)

if __name__ == "__main__":
    http = WSGIServer(('', 5000), app)
    http.serve_forever()